<img src="./Assets/1.1 AgWorkHeaderImage.png">

## Multi-step Agentic Flow

<img src="./Assets/3.1 MultiStep.png">

---

#### Load All Keys

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

True

## Recall: AstraDB Vector Tool (Custom CrewAI Tools)

In [4]:
Token=os.environ["ASTRA_TOKEN"]
EndPoint=os.environ["ASTRA_ENDPOINT"]

In [2]:
from astrapy import DataAPIClient
from astrapy.data_types import DataAPIVector
from openai import OpenAI

def search_documents(query: str, top_k: int = 5) -> list[str]:
    # Set up OpenAI
    openai_client = OpenAI()
    query_embedding = openai_client.embeddings.create(
        model="text-embedding-3-small",
        input=query,
        encoding_format="float"
    ).data[0].embedding

    # Connect to Astra
    astra_client = DataAPIClient(Token)
    db = astra_client.get_database_by_api_endpoint(EndPoint)
    collection = db.get_collection("CollectionSmallEmbeddings")

    # Query Astra
    cursor = collection.find(
        sort={"$vector": DataAPIVector(query_embedding)},
        limit=top_k,
        include_similarity=True
    )

    # Return documents
    results = []
    for doc in cursor:
        text = doc.get("text") or str(doc)
        score = doc.get("$similarity", 0)
        results.append(f"[{score:.4f}] {text}")
    return results


In [6]:
from crewai.tools import BaseTool

class AstraSearchTool(BaseTool):
    name: str = "Vector Tool"
    description: str = "Search the vector store for information about employee leave policy of AKAIWorks"

    def _run(self, query: str) -> str:
        # This is where your vector search logic goes
        results = search_documents(query)
        return "\n".join(results)

In [7]:
astra_tool = AstraSearchTool()

---

### Multi-Step Agent

In [73]:
from crewai import LLM

llm = LLM(
    model="gpt-4o-mini",
    temperature=0.8,
    max_tokens=150,
    top_p=0.9,
    frequency_penalty=0.1,
    presence_penalty=0.1,
    stop=["END"],
    seed=42
)

In [74]:
from crewai import Agent
from crewai_tools import SerperDevTool

multi_reason_agent = Agent(
    role="Advanced Reasoning AI Assistant",
    goal="Help users by either answering their queries directly using built-in knowledge,"
        " or use the VectorTool when the query is about employee leave policy of AKAIWorks "
        "or performing a live web search using the Google Serper API when up-to-date or external information is required."
        "Note that you can also use more than one methods together.",

    backstory="You were built to bridge the gap between real-time information and artificial intelligence. "
            "Equipped with access to the web through the Serper API, employee leave policy via the vector tool, you excel at retrieving the most recent, relevant data when needed."
            " But you also know when to keep things simple — using your own understanding to answer straightforward questions without delay."
            " You’re the perfect blend of real-time access and inherent intelligence. "
            "Remember that company name might not be explicitly mentioned but assume that the leave policy query is implicitly for the organisation.""",
    tools=[SerperDevTool(),astra_tool],
    verbose=True,
    llm=llm
)


In [75]:
from crewai import Task

## Classification -> Validation -> Retrieval -> Combination -> Generation -> Export

#### Task #1: Classification

In [76]:
classify_task = Task(
    description=(
        "Analyze the user query: '{user_input}' and determine which sources are needed to answer it. "
        "Always return more than one : [WEB], [VECTOR], [LLM]. "
        "If the query is simple or general knowledge (e.g., definitions, common facts), "
        "classify as LLM to use internal knowledge. "
        "If the query appears to require current or web-based information "
        "(e.g., news, trending topics, recent data), "
        "classify as WEB to search using the Serper API tool to gather relevant information first. "
        "If the query is about the employee leave policy of AKAIWorks, "
        "classify as VECTOR to perform a vector database search using vector search tool."
        "Remember that company name might not be explicitly mentioned but assume that the leave policy query is implicitly for the organisation."    ),
    expected_output="A list like [WEB], [LLM] or [VECTOR]",
    agent=multi_reason_agent
)

In [77]:
reflect_task = Task(
    description=(
        "You previously classified the sources required. "
        "Reflect carefully — should you add or remove any source? "
        "Return a refined list"
        ),
    expected_output="A revised list of: [WEB], [VECTOR], [LLM]",
    agent=multi_reason_agent
)

#### Task #3: Retrieval

In [78]:
retrieve_task = Task(
    description=(
        "Retrieve the relevant information based on your refined classification. "
        "Use the Serper tool for [WEB], the vector tool for [VECTOR], or internal LLM knowledge for [LLM]. "
        "Summarize each source's contribution. Output should look like:\n"
        "WEB: ...\nVECTOR: ...\nLLM: ..."
    ),
    expected_output="A dictionary-style summary like: WEB:..., VECTOR:..., LLM:...",
    agent=multi_reason_agent,
    tools=[SerperDevTool(),astra_tool]
)

#### Task #4: Combine

In [79]:
combine_task = Task(
    description=(
        "You've gathered data from one or more sources. "
        "Combine the information into a coherent, unified summary. "
        "Resolve contradictions, emphasize relevance, and organize the content logically."
    ),
    expected_output="A combined, logically cohesive summary of all source content.",
    agent=multi_reason_agent
)

#### Task #5: Synthesis

In [80]:
synthesize_task = Task(
    description=(
        "Write the final response to the original query: '{user_input}'. "
        "Make it helpful, complete, and readable. "
        "At the end, note which sources were used."
    ),
    expected_output="The final answer to the user query, including a reference to source(s).",
    agent=multi_reason_agent
)

#### Task #6: Export

In [81]:
export_task = Task(
    description=(
        "Evaluate your final answer. If it's longer than 300 words or contains valuable insights, "
        "write it to a file named 'response.md' in markdown format. Include the query and answer. "
        "If it’s not important enough, skip saving. In either case, return a message saying what happened."
    ),
    expected_output="Saved to markdown OR Skipped saving. Also return the final answer to the user query, including a reference to source(s).",
    agent=multi_reason_agent,
    output_file="response.md"
)

In [82]:
from crewai import Crew, Process

crew = Crew(
    agents=[multi_reason_agent],
    tasks=[
        classify_task, # CoT
        reflect_task, # Reflexion
        retrieve_task, # ReAct
        combine_task, # CoT
        synthesize_task, # CoT
        export_task # Reflexion + PAL
    ],
    process=Process.sequential
)


In [83]:
result = crew.kickoff(inputs={"user_input": "How many leaves can the employees take?"})

# Agent: Advanced Reasoning AI Assistant
## Task: Analyze the user query: 'How many leaves can the employees take?' and determine which sources are needed to answer it. Always return more than one : [WEB], [VECTOR], [LLM]. If the query is simple or general knowledge (e.g., definitions, common facts), classify as LLM to use internal knowledge. If the query appears to require current or web-based information (e.g., news, trending topics, recent data), classify as WEB to search using the Serper API tool to gather relevant information first. If the query is about the employee leave policy of AKAIWorks, classify as VECTOR to perform a vector database search using vector search tool.Remember that company name might not be explicitly mentioned but assume that the leave policy query is implicitly for the organisation.




# Agent: Advanced Reasoning AI Assistant
## Thought: The user query is specifically about the leave policy, which relates to employee leave entitlements. This indicates that the information needed is likely to be specific to AKAIWorks' policies.
## Using tool: Vector Tool
## Tool Input: 
"{\"query\": \"How many leaves can the employees take?\"}"
## Tool Output: 
[0.7726] {'_id': 'a213f2c3a6a042dfae0555a613363f49', 'page_content': 'to\n \nprovide\n \na\n \ncomprehensive\n \nleave\n \nprogram\n \nthat\n \nmeets\n \nthe\n \nneeds\n \nof\n \nour\n \nworkforce.\n \nWhether\n \nan\n \nemployee\n \nis\n \ntaking\n \ntime\n \noff\n \nfor\n \nillness,\n \nvacation,\n \npersonal\n \nmatters,\n \nor\n \nother\n \ncircumstances,\n \nthis\n \npolicy\n \nclarifies\n \nthe\n \navailable\n \nleave\n \noptions\n \nand\n \nthe\n \nprocedures\n \nfor\n \nrequesting\n \nand\n \ntaking\n \nleave.\n \nThe  policy  provides  guidance  on:  \n●  Types  of  leave  available  ●  How  leave  is  accrued  and  

In [84]:
from IPython.display import Markdown, display
display(Markdown("### "+result.raw))

### ### Employee Leave Policy of AKAIWorks LLP

The Employee Leave Policy of AKAIWorks LLP is designed to outline the procedures, expectations, and entitlements regarding time off from work for all employees. This policy ensures that employees have the opportunity to take necessary leave for personal, medical, or family reasons while maintaining the smooth operation of the organization. It is important that all employees understand and adhere to the guidelines within this document to foster a fair and transparent process for all parties involved.

#### Key Aspects of the Leave Policy:

1. **Purpose**: The policy aims to provide clear procedures, expectations, and entitlements related to time off work. It emphasizes the importance of

In [86]:
result.token_usage

UsageMetrics(total_tokens=78808, prompt_tokens=77460, cached_prompt_tokens=23808, completion_tokens=1348, successful_requests=16)